In [3]:
!pip install ultralytics
!pip install wget


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from pycocotools.coco import COCO # wrapper to get data from coco dataset
import skimage.io as io
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random
import os
import wget
from ultralytics import YOLO


In [5]:
df = pd.read_csv('person_anns.csv')

In [6]:
img_url_list = list(df['img_url'].unique())
print(len(img_url_list))

58883


In [71]:
random.seed(24)
img_sample = random.sample(img_url_list, 5)
print(len(img_sample))

5


In [72]:

# Scarica tutte le immagini dalla lista
old_cwd = os.getcwd()
os.chdir(old_cwd+'/imgs')
for url in img_sample:
    try:
        wget.download(url)
    except Exception as e:
        print(f"\nErrore nel download di {url}: {e}")
os.chdir(old_cwd)

In [73]:

# Carica il modello pre-addestrato (puoi scegliere altri modelli come 'yolov8n.pt', 'yolov10.pt', ecc.)
model = YOLO('yolov8n.pt')  # Usa il modello YOLOv8 Nano pre-addestrato


In [75]:
result = model.predict(source="./imgs", save=True)



image 1/6 c:\Users\fedep\OneDrive\Desktop\Universita'\Year 2\PER\FemaleObjectificationInMovies\imgs\000000061822.jpg: 384x640 8 persons, 4 cars, 1 kite, 107.4ms
image 1/6 c:\Users\fedep\OneDrive\Desktop\Universita'\Year 2\PER\FemaleObjectificationInMovies\imgs\000000061822.jpg: 384x640 8 persons, 4 cars, 1 kite, 107.4ms
image 2/6 c:\Users\fedep\OneDrive\Desktop\Universita'\Year 2\PER\FemaleObjectificationInMovies\imgs\000000091057.jpg: 640x480 1 dog, 1 pizza, 209.7ms
image 2/6 c:\Users\fedep\OneDrive\Desktop\Universita'\Year 2\PER\FemaleObjectificationInMovies\imgs\000000091057.jpg: 640x480 1 dog, 1 pizza, 209.7ms
image 3/6 c:\Users\fedep\OneDrive\Desktop\Universita'\Year 2\PER\FemaleObjectificationInMovies\imgs\000000124615.jpg: 544x640 4 persons, 2 couchs, 4 remotes, 192.9ms
image 3/6 c:\Users\fedep\OneDrive\Desktop\Universita'\Year 2\PER\FemaleObjectificationInMovies\imgs\000000124615.jpg: 544x640 4 persons, 2 couchs, 4 remotes, 192.9ms
image 4/6 c:\Users\fedep\OneDrive\Desktop\Un

modificare img_id per tutte le entries

In [ ]:
def confidence_score(result_img):
    detection_list = []
    for res in result_img:
        detection_dict = {}
        img_id = res.path.split('\\')[-1].strip('.jpg').lstrip('0')
        detection_dict['Img_ID'] = img_id
        # Get mask for class 0
        class_mask = res.boxes.cls == 0
        
        # Get all relevant data in one go
        confidences = res.boxes.conf[class_mask]
        dimensions = res.boxes.xywhn[class_mask]
        
        # Print results using zip to iterate both arrays simultaneously
        for index, (conf, dim) in enumerate(zip(confidences, dimensions)):
            detection_dict['Confidence'] = conf.item()
            detection_dict['Dimensions'] = dim
            detection_dict['ID'] = f'{img_id}{index}'
            detection_list.append(detection_dict)    
    print(detection_list)    

In [80]:
print(result[1].path)

c:\Users\fedep\OneDrive\Desktop\Universita'\Year 2\PER\FemaleObjectificationInMovies\imgs\000000091057.jpg


In [105]:
confidence_score(result)

[{'Img_ID': '61822', 'Confidence': 0.2536027431488037, 'Dimensions': tensor([0.4707, 0.6867, 0.0142, 0.0801]), 'ID': '618227'}, {'Img_ID': '91057'}, {'Img_ID': '124615', 'Confidence': 0.6433322429656982, 'Dimensions': tensor([0.7411, 0.8848, 0.2185, 0.2234]), 'ID': '1246153'}, {'Img_ID': '203201', 'Confidence': 0.9247285723686218, 'Dimensions': tensor([0.3509, 0.6135, 0.4268, 0.6710]), 'ID': '2032010'}, {'Img_ID': '257297', 'Confidence': 0.8036516904830933, 'Dimensions': tensor([0.9371, 0.5801, 0.1255, 0.5948]), 'ID': '2572973'}, {'Img_ID': '544264', 'Confidence': 0.8527050018310547, 'Dimensions': tensor([0.7203, 0.5075, 0.2557, 0.3685]), 'ID': '5442641'}]
